In [1]:
import torch

# Preprocessing

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,LabelEncoder

In [27]:
df=pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [28]:
df.shape

(569, 33)

In [29]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [30]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [31]:
xtrain,xtest,ytrain,ytest=train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)

In [32]:
scaler=StandardScaler()
xtrain=scaler.fit_transform(xtrain)
xtest=scaler.transform(xtest)

In [33]:
ytrain[:10]

84     B
127    M
310    B
133    B
458    B
10     M
464    B
102    B
424    B
93     B
Name: diagnosis, dtype: object

In [34]:
encoder=LabelEncoder()
ytrain=encoder.fit_transform(ytrain)
ytest=encoder.transform(ytest)

In [35]:
ytrain[:10]

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0])

In [36]:
xtraintensor=torch.from_numpy(xtrain)
xtesttensor=torch.from_numpy(xtest)
ytraintensor=torch.from_numpy(ytrain)
ytesttensor=torch.from_numpy(ytest)

In [37]:
xtraintensor.shape

torch.Size([455, 30])

# Neural Netork part

In [44]:
class MySimpleNN():
    def __init__(self,x):
        self.weights=torch.rand(x.shape[1],1,dtype=torch.float64,requires_grad=True)
        self.bias=torch.zeros(1,dtype=torch.float64,requires_grad=True)

    def forward(self,x):
        z=torch.matmul(x,self.weights)+self.bias
        return torch.sigmoid(z)

    def lossfunction(self,ypred,y):
        eps=1e-7
        ypred=torch.clamp(ypred,eps,1-eps)
        loss=-(ytraintensor*torch.log(ypred)+(1-ytraintensor)*torch.log(1-ypred)).mean()
        return loss

In [45]:
lr=0.1
epochs=25

In [46]:
model=MySimpleNN(xtraintensor)

In [50]:
for epoch in range(epochs):
    ypred=model.forward(xtraintensor)
    loss=model.lossfunction(ypred,ytraintensor)
    loss.backward()
    with torch.no_grad():
        model.weights-=lr*model.weights.grad
        model.bias-=lr*model.bias.grad
    model.weights.grad.zero_()
    model.bias.grad.zero_()
    print(f'epoch: {epoch+1}, loss: {loss.item()}')   

epoch: 1, loss: 0.9677930445599789
epoch: 2, loss: 0.9349654651928716
epoch: 3, loss: 0.9071453302629653
epoch: 4, loss: 0.8834053593981281
epoch: 5, loss: 0.8629943734102595
epoch: 6, loss: 0.8453108824634769
epoch: 7, loss: 0.8298778988613718
epoch: 8, loss: 0.8163188291209985
epoch: 9, loss: 0.8043356236970424
epoch: 10, loss: 0.7936903559934054
epoch: 11, loss: 0.7841906898867441
epoch: 12, loss: 0.7756789842672459
epoch: 13, loss: 0.7680243730572361
epoch: 14, loss: 0.7611170608089103
epoch: 15, loss: 0.7548641710651905
epoch: 16, loss: 0.7491866515479183
epoch: 17, loss: 0.744016897771676
epoch: 18, loss: 0.7392968752642171
epoch: 19, loss: 0.7349765999362275
epoch: 20, loss: 0.7310128861900164
epoch: 21, loss: 0.7273683031530424
epoch: 22, loss: 0.7240102982524604
epoch: 23, loss: 0.7209104588758438
epoch: 24, loss: 0.7180438899769928
epoch: 25, loss: 0.7153886899717411


In [51]:
model.weights

tensor([[ 0.4007],
        [ 0.0243],
        [-0.0894],
        [ 0.4345],
        [ 0.0847],
        [-0.0133],
        [-0.0475],
        [-0.4832],
        [ 0.1432],
        [ 0.3125],
        [-0.2213],
        [ 0.3321],
        [-0.2332],
        [-0.1763],
        [-0.0783],
        [ 0.0535],
        [ 0.3157],
        [-0.4264],
        [-0.1093],
        [ 0.3621],
        [ 0.5106],
        [ 0.1903],
        [-0.1922],
        [-0.2240],
        [-0.0211],
        [-0.2000],
        [ 0.1127],
        [ 0.1698],
        [ 0.1541],
        [-0.3220]], dtype=torch.float64, requires_grad=True)

In [52]:
model.bias

tensor([-0.3282], dtype=torch.float64, requires_grad=True)

In [55]:
with torch.no_grad():
    ypred=model.forward(xtesttensor)
    ypred=(ypred>0.8).float()
    acc=(ypred==ytesttensor).float().mean()
    print(f'acc :{acc}')

acc :0.6140350699424744
